In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from boruta import BorutaPy

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

from tqdm import tqdm_notebook, tqdm
tqdm.pandas(tqdm_notebook)

from sklearn.preprocessing import StandardScaler  # For scaling dataset
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation 

from apyori import apriori

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn import svm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split functionn
from IPython.display import Image  
from sklearn import tree

from sklearn.datasets import make_blobs


C:\Users\Public\anaconda\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('Bank_CS.csv') 

FileNotFoundError: [Errno 2] File Bank_CS.csv does not exist: 'Bank_CS.csv'

# Data Preprocessing

In [ ]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']

In [ ]:
df1 = df.copy()
#df1.isna().sum()

In [ ]:
#nearest
df1 = df.copy()
df1['Years_to_Financial_Freedom'] = df1['Years_to_Financial_Freedom'].fillna(method='ffill')
df1['Number_of_Credit_Card_Facility'] = df1['Number_of_Credit_Card_Facility'].fillna(method='ffill')
df1['Number_of_Bank_Products'] = df1['Number_of_Bank_Products'].fillna(method='ffill')
    
df1['Property_Type'] = df1['Property_Type'].fillna(method='ffill')
df1['Years_for_Property_to_Completion'] = df1['Years_for_Property_to_Completion'].fillna(method='ffill')

In [ ]:
#mean
df1['Loan_Amount'] = df1['Loan_Amount'].fillna(round(df1['Loan_Amount'].mean()))
df1['Loan_Tenure_Year'] = df1['Loan_Tenure_Year'].fillna(round(df1['Loan_Tenure_Year'].mean()))
df1['Number_of_Properties'] = df1['Number_of_Properties'].fillna(round(df1['Number_of_Properties'].mean()))

df1['Number_of_Side_Income'] = df1['Number_of_Side_Income'].fillna(0)
df1['Monthly_Salary'] = df1['Monthly_Salary'].fillna(round(df1['Monthly_Salary'].mean()))
df1['Total_Sum_of_Loan'] = df1['Total_Sum_of_Loan'].fillna(round(df1['Total_Sum_of_Loan'].mean()))

df1['Credit_Card_types'] = df1['Credit_Card_types'].fillna('Not Specified')

In [ ]:
df1["State"].unique()

In [ ]:
df1['State']=df1['State'].replace('Johor B','Johor')
df1['State']=df1['State'].replace('Penang','Pulau Pinang')
df1['State']=df1['State'].replace('P.Pinang','Pulau Pinang')
df1['State']=df1['State'].replace('Pulau Penang','Pulau Pinang')
df1['State']=df1['State'].replace('K.L','Kuala Lumpur')
df1['State']=df1['State'].replace('N.S','N.Sembilan')
df1['State']=df1['State'].replace('SWK','Sarawak')
df1['State']=df1['State'].replace('Trengganu','Terengganu')



In [ ]:
df1['Total_Income_for_Join_Application']= np.where(df1['Total_Income_for_Join_Application'] < df1['Monthly_Salary'],  
                                                   df1['Total_Income_for_Join_Application'].fillna(round(df1['Monthly_Salary'])),
                                                   df1['Total_Income_for_Join_Application'].fillna(round(df1['Total_Income_for_Join_Application'].mean())))

In [ ]:
df1.isna().sum() 

In [ ]:
df1['Credit_Card_types'].mode

In [ ]:
#df1.isna().sum() 

In [ ]:
#df1.head(100)

In [ ]:
#df['Number_of_Side_Income']==0

In [ ]:
#df1['Credit_Card_types'].unique()

df1.info()

# Exploratory Data Analysis

In [ ]:
df1["Decision"].value_counts().plot.bar()
df1["Decision"].value_counts()
#75% Accepted from 2350 Applicant (1769)
#581 rejected

In [ ]:
plt.figure(1)
plt.subplot(321)
df1["Employment_Type"].value_counts(normalize=True).plot.bar(figsize=(20,10), title= "Employment")
plt.subplot(322)
df1["Credit_Card_types"].value_counts(normalize=True).plot.bar(figsize=(20,10), title= "Credit_Card_types")

plt.subplot(325)
df1["Property_Type"].value_counts(normalize=True).plot.bar(figsize=(20,10), title= "Property_Type")
plt.subplot(326)
df1["State"].value_counts(normalize=True).plot.bar(figsize=(20,10), title= "State")
plt.show()

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(df1['Monthly_Salary']);
plt.subplot(122) 
df1['Monthly_Salary'].plot.box(figsize=(16,5))

plt.show()


#graph for monthly_salary is normally distributed

In [ ]:
df1.boxplot(column = "Monthly_Salary",by= "Employment_Type") 
plt.suptitle("")

#There is employer that having very high income and appear to be outliers

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(df1['Loan_Amount']);
plt.subplot(122) 
df1['Loan_Amount'].plot.box(figsize=(16,5))

plt.show()
#mean of loan amount applied is around 400000
#no outliers

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(df1['Total_Income_for_Join_Application']);
plt.subplot(122) 
df1['Total_Income_for_Join_Application'].plot.box(figsize=(16,10))

plt.show()


#great numbers of outliers

# Bivariate

In [ ]:
heatmap = df1.copy()
corr1 = heatmap.corr()

plt.figure(figsize=(20,20))
sns.heatmap(corr1, vmax=.8, square=True, annot=True, fmt= '.2f', annot_kws={'size': 15}, cmap="BuPu")

In [ ]:
f2 = df1.groupby('Employment_Type')['Decision'].value_counts()

f2.unstack(0).plot.bar(figsize=(20,15))
#plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.title("Frequency of Credit Credit Card Type in Each State")

In [ ]:
f2 = df1.groupby('Credit_Card_Exceed_Months')['Decision'].value_counts()

f2.unstack(0).plot.bar(figsize=(20,15))
#plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.title("Frequency of Credit Credit Card Type in Each State")

In [ ]:
f2 = df1.groupby('Credit_Card_types')['State'].value_counts()

f2.unstack(0).plot.bar(figsize=(20,15))
#plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.title("Frequency of Credit Credit Card Type in Each State")

In [ ]:
filter1 = df1[df1["Decision"] == 'Accept']
filter1 = filter1.groupby(["State"], as_index=False).agg({"Monthly_Salary": "mean"})
filter1 = filter1.rename(columns={"Monthly_Salary": "Monthly_SalaryYes"})

filter2 = df1[df1["Decision"] == 'Reject']
filter2 = filter2.groupby(["State"], as_index=False).agg({"Monthly_Salary": "mean"})
filter2 = filter2.rename(columns={"Monthly_Salary": "Monthly_SalaryNo"})

filter9 = filter1.merge(filter2,on="State",how='inner')

In [ ]:
filter9

In [ ]:
filtered = df1[df1["Decision"] == "Accept"]
filtered= (filtered.groupby(["State"]).agg({"Monthly_Salary": "mean"}))
f1 = filtered.values

filteredN = df1[df1["Decision"] == "Reject"]
filteredN= (filtered.groupby(["State"]).agg({"Monthly_Salary": "mean"}))
f2 = filtered.values

labels = filter9["State"]
YES = filter9["Monthly_SalaryYes"]
NO = filter9["Monthly_SalaryNo"]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(20,15))
rects1 = ax.bar(x - width/2, YES, width, label='Accept')
rects2 = ax.bar(x + width/2, NO, width, label='Reject')

#ax.set_ylabel('Scores')
#ax.set_title('Scores by group and gender')
ax.set_xticks(x)

ax.set_xticklabels(labels, rotation=90)
ax.legend()

plt.show()

In [ ]:
filter1 = df1[df1["Decision"] == 'Accept']
filter1 = filter1.groupby(["Property_Type"], as_index=False).agg({"Monthly_Salary": "mean"})
filter1 = filter1.rename(columns={"Monthly_Salary": "Monthly_SalaryYes"})

filter2 = df1[df1["Decision"] == 'Reject']
filter2 = filter2.groupby(["Property_Type"], as_index=False).agg({"Monthly_Salary": "mean"})
filter2 = filter2.rename(columns={"Monthly_Salary": "Monthly_SalaryNo"})

filter9a = filter1.merge(filter2,on="Property_Type",how='inner')
filter9a




In [ ]:
labels = filter9a["Property_Type"]
PropertyYes = filter9a["Monthly_SalaryYes"]
PropertyNo = filter9a["Monthly_SalaryNo"]


x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(20,15))
rects1 = ax.bar(x - width/2, PropertyYes, width, label='Accept')
rects2 = ax.bar(x + width/2, PropertyNo, width, label='Reject')

#ax.set_ylabel('Scores')
#ax.set_title('Scores by group and gender')
ax.set_xticks(x)

ax.set_xticklabels(labels, rotation=90)
ax.legend()

plt.show()


In [ ]:
#count -> no of properties , 

f3 = df1.groupby('Number_of_Properties')['Decision'].value_counts()

f3.unstack(0).plot.bar(figsize=(20,15))
#plt.legend(loc='center left',bbox_to_anchor=(1.0,0.5))
plt.title("Frequency of Credit Credit Card Type in Each State")

New Graph

In [ ]:
fa1= df1[["Number_of_Properties","Decision","Credit_Card_types","Employment_Type"]]
g1= (fa1["Decision"]== 'Accept')
g2=fa1[g1]

g3=((g2["Credit_Card_types"]=='platinum') 
    | (g2["Credit_Card_types"]=='gold')
    | (g2["Credit_Card_types"]=='normal')
    | (g2["Credit_Card_types"]=='Not Specified'))

g5=g2[g3]
g5

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Employment_Type", hue = 'Credit_Card_types', data=g5)
plt.show()

In [ ]:
fa1= df1[["Number_of_Properties","Decision","Credit_Card_types","Employment_Type"]]
g1= (fa1["Decision"]== 'Reject')
g2=fa1[g1]

g3=((g2["Credit_Card_types"]=='platinum') 
    | (g2["Credit_Card_types"]=='gold')
    | (g2["Credit_Card_types"]=='normal')
    | (g2["Credit_Card_types"]=='Not Specified'))

g5=g2[g3]
g5

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Employment_Type", hue = 'Credit_Card_types', data=g5)
plt.show()

In [ ]:
nodependent= df1[["Number_of_Dependents","Total_Sum_of_Loan","Monthly_Salary","Decision"]]
nodependent["Salary_Group"] = pd.cut(nodependent["Monthly_Salary"], bins = [3000,5000,7000,9000,11000,13000])

a1= (nodependent["Decision"]== 'Accept')
a2=nodependent[a1]

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Salary_Group", hue = 'Number_of_Dependents', data=a2)
plt.show()

In [ ]:
d1= (nodependent["Decision"]== 'Reject')
d2=nodependent[d1]

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Salary_Group", hue = 'Number_of_Dependents', data=d2)
plt.show()

In [ ]:
sumofloan = df1[["Number_of_Dependents","Total_Sum_of_Loan","Monthly_Salary","Decision"]]

sumofloan["Salary_Group"] = pd.cut(sumofloan["Monthly_Salary"], bins = [3000,5000,7000,9000,11000,13000])
sumofloan["Sum_of_Loan_Group"] = pd.cut(sumofloan["Total_Sum_of_Loan"], bins = [400000,600000,800000,1000000,1200000,1500000])

b1= (sumofloan["Decision"]== 'Accept')
b2=sumofloan[b1]

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Salary_Group", hue = 'Sum_of_Loan_Group', data=b2)
plt.show()

In [ ]:
c1= (sumofloan["Decision"]== 'Reject')
c2=sumofloan[c1]

sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.countplot( x="Salary_Group", hue = 'Sum_of_Loan_Group', data=c2)
plt.show()

# Label Encode

In [ ]:
df2= df1.copy()

In [ ]:
df2['Employment_Type'] = LabelEncoder().fit_transform(df2.Employment_Type)
df2['More_Than_One_Products'] = LabelEncoder().fit_transform(df2.More_Than_One_Products)
df2['Credit_Card_types'] = LabelEncoder().fit_transform(df2.Credit_Card_types)
df2['Property_Type'] = LabelEncoder().fit_transform(df2.Property_Type)
df2['State'] = LabelEncoder().fit_transform(df2.State)
df2['Decision'] = LabelEncoder().fit_transform(df2.Decision)


In [ ]:
y = df2.Decision
X = df2.drop("Decision",1)
colnames = X.columns
X.shape


# Boruta

In [ ]:
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
feat_selector = BorutaPy(rf, n_estimators="auto", random_state=1)
feat_selector.fit(X.values, y.values.ravel())


# sort the boruta score descending

boruta_score = ranking(list(map(float, feat_selector.ranking_)), colnames, order=-1)
boruta_score = pd.DataFrame(list(boruta_score.items()), columns=['Features', 'Score'])
boruta_score = boruta_score.sort_values('Score', ascending=False)

In [ ]:
print('---------Top 10----------')
display(boruta_score.head(10))

print('---------Bottom 10----------')   
boruta_score.tail(10)

In [ ]:
sns_boruta_plot = sns.catplot(x="Score", y="Features", data = boruta_score[0:30], kind = "bar", 
               height=14, aspect=1.9, palette='coolwarm')
plt.title("Boruta Top 20 Features")

# Classification model before SMOTE

In [ ]:
Xtr, Xts, ytr, yts = train_test_split(X,y.values.ravel(),test_size=0.3,random_state=10)

In [ ]:
Xtr=Xtr[['Loan_Amount','Monthly_Salary','Total_Sum_of_Loan','State','Credit_Card_types','Number_of_Credit_Card_Facility','Employment_Type','Credit_Card_Exceed_Months','Total_Income_for_Join_Application','Loan_Tenure_Year','Years_to_Financial_Freedom']]
Xts=Xts[['Loan_Amount','Monthly_Salary','Total_Sum_of_Loan','State','Credit_Card_types','Number_of_Credit_Card_Facility','Employment_Type','Credit_Card_Exceed_Months','Total_Income_for_Join_Application','Loan_Tenure_Year','Years_to_Financial_Freedom']]

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtr,ytr)

y_pred = knn.predict(Xts)


print("Accuracy on training set: {:.3f}".format(knn.score(Xtr, ytr)))
print("Accuracy on test set: {:.3f}".format(knn.score(Xts, yts)))
#knn.score(X_test,y_test)

confusion_majority=confusion_matrix(yts, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(yts, y_pred)))
print('Recall= {:.2f}'. format(recall_score(yts, y_pred)))
print('F1= {:.2f}'. format(f1_score(yts, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(yts, y_pred)))

# Decision Tree

In [ ]:
DT = DecisionTreeClassifier(criterion='entropy',max_depth=12)
DT.fit(Xtr,ytr)
y_pred=DT.predict(Xts)

confusion_majority=confusion_matrix(yts, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(yts, y_pred)))
print('Recall= {:.2f}'. format(recall_score(yts, y_pred)))
print('F1= {:.2f}'. format(f1_score(yts, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(yts, y_pred)))

In [ ]:
DT = DecisionTreeClassifier(criterion='entropy',max_depth=12)

kf = KFold(n_splits=5,shuffle=True)
kf.split(Xtr)

accuracy_model = []
i=0
for train_index,test_index in kf.split(Xtr):
    
    Xtrain, Xtest = Xtr.iloc[train_index], Xtr.iloc[test_index]
    ytrain, ytest = ytr[train_index], ytr[test_index]
    DT.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, DT.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

# Random Forest

In [ ]:
forest = RandomForestClassifier(max_depth = 10)
forest.fit(Xtr,ytr)
y_pred=forest.predict(Xts)
confusion_majority=confusion_matrix(yts, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(yts, y_pred)))
print('Recall= {:.2f}'. format(recall_score(yts, y_pred)))
print('F1= {:.2f}'. format(f1_score(yts, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(yts, y_pred)))

In [ ]:
forest = RandomForestClassifier(max_depth = 10)

kf = KFold(n_splits=5,shuffle=True)
kf.split(Xtr)

accuracy_model = []
i=0
for train_index,test_index in kf.split(Xtr):
    
    Xtrain, Xtest = Xtr.iloc[train_index], Xtr.iloc[test_index]
    ytrain, ytest = ytr[train_index], ytr[test_index]
    forest.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, forest.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()


# XGBoost

In [ ]:
xgb1=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.9,
                     colsample_bytree=0.95,reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xgb1.fit(Xtr,ytr)
y_pred=xgb1.predict(Xts)

confusion_majority=confusion_matrix(yts, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(yts, y_pred)))
print('Recall= {:.2f}'. format(recall_score(yts, y_pred)))
print('F1= {:.2f}'. format(f1_score(yts, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(yts, y_pred)))

In [ ]:
xgb1=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.9,
                     colsample_bytree=0.95,reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

kf = KFold(n_splits=5,shuffle=True)
kf.split(Xtr)

accuracy_model = []
i=0
for train_index,test_index in kf.split(Xtr):
    
    Xtrain, Xtest = Xtr.iloc[train_index], Xtr.iloc[test_index]
    ytrain, ytest = ytr[train_index], ytr[test_index]
    xgb1.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, xgb1.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

# Adaboost RF

In [ ]:
ada = AdaBoostClassifier(base_estimator=forest ,n_estimators=500, learning_rate=0.1, random_state=0)
ada.fit(Xtr,ytr)
y_pred=ada.predict(Xts)

confusion_majority=confusion_matrix(yts, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(yts, y_pred)))
print('Recall= {:.2f}'. format(recall_score(yts, y_pred)))
print('F1= {:.2f}'. format(f1_score(yts, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(yts, y_pred)))

In [ ]:
ada = AdaBoostClassifier(base_estimator=forest ,n_estimators=500, learning_rate=0.1, random_state=0)

kf = KFold(n_splits=5,shuffle=True)
kf.split(Xtr)

accuracy_model = []
i=0
for train_index,test_index in kf.split(Xtr):
    
    Xtrain, Xtest = Xtr.iloc[train_index], Xtr.iloc[test_index]
    ytrain, ytest = ytr[train_index], ytr[test_index]
    ada.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, ada.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

# Smote

In [ ]:
os = SMOTE(random_state=10)
#X_train, X_test, y_train, y_test = train_test_split(X,y.values.ravel(),test_size=0.3,random_state=10)
columns = X.columns
os_data_X, os_data_y=os.fit_sample(X,y)

os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
os_data_y = pd.DataFrame(data=os_data_y,columns=['Decision'])

print("length of oversampled data is",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['Decision']==0]))
print("Number of subscription",len(os_data_y[os_data_y['Decision']==1]))
print("Proportion of no subscription data in oversampled data is",len(os_data_y[os_data_y['Decision']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is",len(os_data_y[os_data_y['Decision']==1])/len(os_data_X))


In [ ]:
#print(df2["Decision"].value_counts())

plt.figure(1)
plt.subplot(121)
df2["Decision"].value_counts().plot(kind="bar",figsize=(16,5))
plt.title("Class Distribution Before Smote")

plt.subplot(122) 
#print(os_data_y["Decision"].value_counts())
os_data_y["Decision"].value_counts().plot(kind="bar",figsize=(16,5))
plt.title("Class Distribution After Smote")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(os_data_X,os_data_y.values.ravel(),test_size=0.3,random_state=10)

In [ ]:
X_train=X_train[['Loan_Amount','Monthly_Salary','Total_Sum_of_Loan','State','Credit_Card_types','Number_of_Credit_Card_Facility','Employment_Type','Credit_Card_Exceed_Months','Total_Income_for_Join_Application','Loan_Tenure_Year','Years_to_Financial_Freedom']]
X_test=X_test[['Loan_Amount','Monthly_Salary','Total_Sum_of_Loan','State','Credit_Card_types','Number_of_Credit_Card_Facility','Employment_Type','Credit_Card_Exceed_Months','Total_Income_for_Join_Application','Loan_Tenure_Year','Years_to_Financial_Freedom']]

# Boruta After SMOTE

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
feat_selector = BorutaPy(rf, n_estimators="auto", random_state=1)
feat_selector.fit(os_data_X.values, os_data_y.values.ravel())


# sort the boruta score descending

boruta_score = ranking(list(map(float, feat_selector.ranking_)), colnames, order=-1)
boruta_score = pd.DataFrame(list(boruta_score.items()), columns=['Features', 'Score'])
boruta_score = boruta_score.sort_values('Score', ascending=False)

In [ ]:
print('---------Top 10----------')
display(boruta_score.head(10))

print('---------Bottom 10----------')   
boruta_score.tail(10)

In [ ]:
sns_boruta_plot = sns.catplot(x="Score", y="Features", data = boruta_score[0:30], kind = "bar", 
               height=14, aspect=1.9, palette='coolwarm')
plt.title("Boruta Top 20 Features")

# Machine Learning Model After Smote

# 1.Random Forest

In [ ]:
forest = RandomForestClassifier(max_depth = 10)
forest.fit(X_train,y_train)
y_pred = forest.predict(X_test)
#scores = cross_val_score(forest, X_train,y_train, cv=5)
#forest.score(X_test, y_test)
 

print("Accuracy on training set: {:.3f}".format(forest.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(X_test, y_test)))
#print(accuracy_score(y_train,scores))

confusion_majority=confusion_matrix(y_test, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(y_test, y_pred)))
print('Recall= {:.2f}'. format(recall_score(y_test, y_pred)))
print('F1= {:.2f}'. format(f1_score(y_test, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(y_test, y_pred)))

# RF crossValidation

In [ ]:
forest = RandomForestClassifier(max_depth = 10)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X_train)

accuracy_model = []
i=0
for train_index,test_index in kf.split(X_train):
    
    Xtrain, Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    forest.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, forest.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

In [ ]:
prob_forest = forest.predict_proba(X_test)
prob_forest = prob_forest[:,1]

auc_forest = roc_auc_score(y_test,prob_forest)
#print('AUC: %.2f'%auc_DT)
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_test, prob_forest) 

plt.plot(fpr_forest, tpr_forest, color='blue', label='RF') 
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

# 2. Adaboost Classifier

In [ ]:
ada = AdaBoostClassifier(base_estimator=forest ,n_estimators=500, learning_rate=0.1, random_state=0)
ada.fit(X_train,y_train)
y_pred = ada.predict(X_test)
ada.score(X_test, y_test)

confusion_majority=confusion_matrix(y_test, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(y_test, y_pred)))
print('Recall= {:.2f}'. format(recall_score(y_test, y_pred)))
print('F1= {:.2f}'. format(f1_score(y_test, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(y_test, y_pred)))

# Adaboost CrossValidation

In [ ]:
ada = AdaBoostClassifier(base_estimator=forest ,n_estimators=500, learning_rate=0.1, random_state=0)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X_train)

accuracy_model = []
i=0
for train_index,test_index in kf.split(X_train):
    
    Xtrain, Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    ada.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, ada.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

In [ ]:
prob_ada = ada.predict_proba(X_test)
prob_ada = prob_ada[:,1]

auc_ada = roc_auc_score(y_test,prob_ada)
#print('AUC: %.2f'%auc_DT)
fpr_ada, tpr_ada, thresholds_ada = roc_curve(y_test, prob_ada) 

plt.plot(fpr_ada, tpr_ada, color='blue', label='ada') 
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

# 3. XGboost classifier

In [ ]:
xgb1=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.9,
                     colsample_bytree=0.95,reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xgb1.fit(X_train,y_train)
y_pred = xgb1.predict(X_test)
xgb1.score(X_test, y_test)

confusion_majority=confusion_matrix(y_test, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(y_test, y_pred)))
print('Recall= {:.2f}'. format(recall_score(y_test, y_pred)))
print('F1= {:.2f}'. format(f1_score(y_test, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(y_test, y_pred)))

# XGB Cross Validation

In [ ]:
xgb1=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.9,
                     colsample_bytree=0.95,reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X_train)

accuracy_model = []
i=0
for train_index,test_index in kf.split(X_train):
    
    Xtrain, Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    xgb1.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, xgb1.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

In [ ]:
prob_xgb1 = xgb1.predict_proba(X_test)
prob_xgb1 = prob_xgb1[:,1]

auc_xgb1 = roc_auc_score(y_test,prob_xgb1)
#print('AUC: %.2f'%auc_DT)
fpr_xgb1, tpr_xgb1, thresholds_xgb1 = roc_curve(y_test, prob_xgb1) 

plt.plot(fpr_xgb1, tpr_xgb1, color='blue', label='xgb') 
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

# 4. Decision Tree Classifier

In [ ]:
DT = DecisionTreeClassifier(criterion='entropy',max_depth=12)
DT=  DT.fit(X_train,y_train)
y_pred= DT.predict(X_test)

print("Accuracy on training set: {:.3f}".format(DT.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(DT.score(X_test, y_test)))

confusion_majority=confusion_matrix(y_test, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(y_test, y_pred)))
print('Recall= {:.2f}'. format(recall_score(y_test, y_pred)))
print('F1= {:.2f}'. format(f1_score(y_test, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(y_test, y_pred)))

# Decision Tree Cross Validation

In [ ]:
DT = DecisionTreeClassifier(criterion='entropy',max_depth=12)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X_train)

accuracy_model = []
i=0
for train_index,test_index in kf.split(X_train):
    
    Xtrain, Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    DT.fit(Xtrain,ytrain)
    accuracy_model.append(accuracy_score(ytest, DT.predict(Xtest), normalize=True)*100)
    #print([i] "of kfold" + n_splits + accuracy_model[i])
    print('\n{} of kfold {}' .format(i+1,kf.n_splits))
    print(accuracy_model[i])
    i += 1

    
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Kfold 1','Kfold 2','Kfold 3','Kfold 4','Kfold 5'],y="Scores",data=scores)
plt.show()
sns.set()

In [ ]:
prob_DT = DT.predict_proba(X_test)
prob_DT = prob_DT[:,1]

auc_DT = roc_auc_score(y_test,prob_DT)
#print('AUC: %.2f'%auc_DT)
fpr_DT, tpr_DT, thresholds_DT = roc_curve(y_test, prob_DT) 

plt.plot(fpr_DT, tpr_DT, color='blue', label='DT') 
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

# 5. KNN

In [ ]:
#insert = X.iloc[[2345]].values.tolist()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)


print("Accuracy on training set: {:.3f}".format(knn.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(knn.score(X_test, y_test)))
#knn.score(X_test,y_test)

confusion_majority=confusion_matrix(y_test, y_pred)

print('Mjority classifier Confusion Matrix\n', confusion_majority)

print('**********************')
print('Mjority TN= ', confusion_majority[0][0])
print('Mjority FP=',  confusion_majority[0][1])
print('Mjority FN= ', confusion_majority[1][0])
print('Mjority TP= ', confusion_majority[1][1])
print('**********************')

print('Precision= {:.2f}'.format(precision_score(y_test, y_pred)))
print('Recall= {:.2f}'. format(recall_score(y_test, y_pred)))
print('F1= {:.2f}'. format(f1_score(y_test, y_pred)))
print('Accuracy= {:.2f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
prob_knn = knn.predict_proba(X_test)
prob_knn = prob_knn[:,1]

auc_knn = roc_auc_score(y_test,prob_knn)
#print('AUC: %.2f'%auc_DT)
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, prob_knn) 

plt.plot(fpr_knn, tpr_knn, color='blue', label='DT') 
plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

In [ ]:
k_range = range(1,20)
scores = []

# your codes here...
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k, weights='uniform')
    knn.fit(X_train,y_train)
    scores.append(knn.score(X_test,y_test))


plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.title('Accuracy by n_neigbors')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20]);
plt.plot(k_range, scores, color='green', linestyle='dashed', linewidth=1, markersize=5)

# Performance Comparison

In [ ]:
plt.plot(fpr_DT, tpr_DT, color='blue', label='DT')
plt.plot(fpr_knn, tpr_knn, color='red', label='KNN')
plt.plot(fpr_forest, tpr_forest, color='yellow', label='RF')
plt.plot(fpr_xgb1, tpr_xgb1, color='black', label='xgb1')
plt.plot(fpr_ada, tpr_ada, color='darkmagenta', label='ada')

plt.plot([0, 1], [0, 1], color='green', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()

# Association Rule mining

In [ ]:
df3=df1.copy()

In [ ]:
df3.info()

In [ ]:
df3['Bin_Loan_Amount'] = pd.cut(df3['Loan_Amount'], bins=[0,200000,400000,600000,800000])
df3['Bin_Monthly_Salary'] = pd.cut(df3['Monthly_Salary'], bins=[0,4000,8000,12000,16000])
df3['Bin_Total_Income_for_Join_Application'] = pd.cut(df3['Total_Income_for_Join_Application'], bins=[6000,10000,14000,18000,22000])
df3['Bin_Total_Sum_of_Loan'] = pd.cut(df3['Total_Sum_of_Loan'], bins=[300000,600000,900000,1200000,1500000])
df3['Bin_Loan_Tenure_Year'] = pd.cut(df3['Loan_Tenure_Year'], bins=[5,10,15,20,25])
#df3['Bin_Years_to_Financial_Freedom'] = pd.cut(df3['Years_to_Financial_Freedom'], bins=[5,10,15,20])
#df3['Bin_Credit_Card_Exceed_Months'] = pd.cut(df3['Credit_Card_Exceed_Months'], bins=[0,2,4,6,8])

df3['Bin_Loan_Amount']=df3['Bin_Loan_Amount'].astype('str')
df3['Bin_Total_Sum_of_Loan']=df3['Bin_Total_Sum_of_Loan'].astype('str')
df3['Bin_Monthly_Salary']=df3['Bin_Monthly_Salary'].astype('str')
df3['Bin_Total_Income_for_Join_Application']=df3['Bin_Total_Income_for_Join_Application'].astype('str')
#df3['Bin_Credit_Card_Exceed_Months']=df3['Bin_Credit_Card_Exceed_Months'].astype('str')
df3['Bin_Loan_Tenure_Year']=df3['Bin_Loan_Tenure_Year'].astype('str')

In [ ]:
df3['Bin_Loan_Amount']=df3['Bin_Loan_Amount'].replace('(0, 200000]','Loan_Amount(0, 200000]')
df3['Bin_Loan_Amount']=df3['Bin_Loan_Amount'].replace('(200000, 400000]','Loan_Amount(200000, 400000]')
df3['Bin_Loan_Amount']=df3['Bin_Loan_Amount'].replace('(400000, 600000]','Loan_Amount(400000, 600000]')
df3['Bin_Loan_Amount']=df3['Bin_Loan_Amount'].replace('(600000, 800000]','Loan_Amount(600000, 800000]')

df3['Bin_Monthly_Salary']=df3['Bin_Monthly_Salary'].replace('(0, 4000]','Monthly_Salary(0, 4000]')
df3['Bin_Monthly_Salary']=df3['Bin_Monthly_Salary'].replace('(4000, 8000]','Monthly_Salary(4000, 8000]')
df3['Bin_Monthly_Salary']=df3['Bin_Monthly_Salary'].replace('(8000, 12000]','Monthly_Salary(8000, 12000]')
df3['Bin_Monthly_Salary']=df3['Bin_Monthly_Salary'].replace('(12000, 16000]','Monthly_Salary(12000, 16000]')

df3['Bin_Total_Sum_of_Loan']=df3['Bin_Total_Sum_of_Loan'].replace('(300000, 600000]','Total_Sum_of_Loan(300000, 600000]')
df3['Bin_Total_Sum_of_Loan']=df3['Bin_Total_Sum_of_Loan'].replace('(600000, 900000]','Total_Sum_of_Loan(600000, 900000]')
df3['Bin_Total_Sum_of_Loan']=df3['Bin_Total_Sum_of_Loan'].replace('(900000, 1200000]','Total_Sum_of_Loan(900000, 1200000]')
df3['Bin_Total_Sum_of_Loan']=df3['Bin_Total_Sum_of_Loan'].replace('(1200000, 1500000]','Total_Sum_of_Loan(1200000, 1500000]')

df3['Bin_Total_Income_for_Join_Application']=df3['Bin_Total_Income_for_Join_Application'].replace('(6000, 10000]','Total_Income_for_Join_Application(6000, 9000]')
df3['Bin_Total_Income_for_Join_Application']=df3['Bin_Total_Income_for_Join_Application'].replace('(10000, 14000]','Total_Income_for_Join_Application(9000, 12000]')
df3['Bin_Total_Income_for_Join_Application']=df3['Bin_Total_Income_for_Join_Application'].replace('(14000, 18000]','Total_Income_for_Join_Application(12000, 15000]')
df3['Bin_Total_Income_for_Join_Application']=df3['Bin_Total_Income_for_Join_Application'].replace('(18000, 22000]','Total_Income_for_Join_Application(15000, 18000]')

df3['Bin_Loan_Tenure_Year']=df3['Bin_Loan_Tenure_Year'].replace('(5, 10]','Loan_Tenure_Year(5, 10]')
df3['Bin_Loan_Tenure_Year']=df3['Bin_Loan_Tenure_Year'].replace('(15, 20]','Loan_Tenure_Year(15, 20]')
df3['Bin_Loan_Tenure_Year']=df3['Bin_Loan_Tenure_Year'].replace('(20, 25]','Loan_Tenure_Year(20, 25]')
df3['Bin_Loan_Tenure_Year']=df3['Bin_Loan_Tenure_Year'].replace('(10, 15]','Loan_Tenure_Year(10, 15]')


#df3['Bin_Credit_Card_Exceed_Months']=df3['Bin_Credit_Card_Exceed_Months'].replace('(0, 2]','Credit_Card_Exceed_Months(0, 2]')
#df3['Bin_Credit_Card_Exceed_Months']=df3['Bin_Credit_Card_Exceed_Months'].replace('(2, 4]','Credit_Card_Exceed_Months(2, 4]')
#df3['Bin_Credit_Card_Exceed_Months']=df3['Bin_Credit_Card_Exceed_Months'].replace('(4, 6]','Credit_Card_Exceed_Months(4, 6]')
#df3['Bin_Credit_Card_Exceed_Months']=df3['Bin_Credit_Card_Exceed_Months'].replace('(6, 8]','Credit_Card_Exceed_Months(6, 8]')

In [ ]:
df3=df3[['Employment_Type','Credit_Card_types','Bin_Total_Income_for_Join_Application','Bin_Total_Sum_of_Loan','Bin_Monthly_Salary','Bin_Loan_Amount','Property_Type']]

In [ ]:
banks = []
for i in range(0, 2350):
    banks.append([str(df3.values[i,j]) for j in range(0, 7)])

In [ ]:
association_rules = apriori(banks,min_support=0.0040,min_confidence=0.45,min_lift=3,min_length=2)
association_results = list(association_rules)
association_results

In [ ]:
# check the len
len(association_results)

In [ ]:
cnt =0

for item in association_results:
    cnt += 1
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("(Rule " + str(cnt) + ") " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(round(item[1],3)))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(round(item[2][0][2],4)))
    print("Lift: " + str(round(item[2][0][3],4)))
    print("=====================================")

In [ ]:
df1.info()

# Clustering Analysis

In [ ]:
dft = df1.copy() 

dfc = df2.copy()
Xc = dfc.drop("Decision",axis=1)
yc = dfc["Decision"]

Xc = pd.get_dummies(Xc, drop_first=True)
Xc.columns

In [ ]:
km = KMeans(n_clusters=3,random_state=12)
km.fit(Xc)
km.labels_

In [ ]:
dfc2 = df2.copy()
dfc2 = dfc2.drop("Decision",axis=1)
dfc2["Label"] = km.labels_
dfc2.head()

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.scatterplot(dft['Monthly_Salary'],dft['Loan_Amount'],hue=dft['Decision'])
plt.subplot(122) 
sns.scatterplot(dfc2['Monthly_Salary'],dfc2['Loan_Amount'],hue=dfc2['Label'])


In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.scatterplot(dft['Total_Income_for_Join_Application'],dft['Loan_Amount'],hue=dft['Decision'])
plt.subplot(122) 
sns.scatterplot(dfc2['Total_Income_for_Join_Application'],dfc2['Loan_Amount'],hue=dfc2['Label'])

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.scatterplot(dft['Total_Income_for_Join_Application'],dft['Total_Sum_of_Loan'],hue=dft['Decision'])
plt.subplot(122) 
sns.scatterplot(dfc2['Total_Income_for_Join_Application'],dfc2['Total_Sum_of_Loan'],hue=dfc2['Label'])

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
sns.scatterplot(dft['Monthly_Salary'],dft['Total_Sum_of_Loan'],hue=dft['Decision'])
plt.subplot(122)
sns.scatterplot(dfc2['Monthly_Salary'],dfc2['Total_Sum_of_Loan'],hue=dfc2['Label'])


# Silhouette

In [ ]:
from yellowbrick.cluster import silhouette_visualizer
print("Silhouette Score(n=2)= ", silhouette_score(dfc2, dfc2['Label']))


In [ ]:
silhouette_visualizer(KMeans(3, random_state=12), dfc2, colors="yellowbrick")

In [ ]:
#Distortions
distortions = []

for i in range(1,11):
    km = KMeans(
        n_clusters = i, init= "random",
        n_init=10,max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(Xc)
    distortions.append(km.inertia_)
#plot

plt.plot(range(1,11),distortions,marker='o')
plt.xlabel("Number of clusters")
plt.ylabel("Distortion")
plt.show()